# Coding: Language Model based on LSTM

 - Dataset: WikiText-2
 - Model: LSTM/GRU
 - Optimizer: Adam
 - Learning Rate: 0.001
 - Batch Size: 64
 - Epochs: 10
 - Dropout: 0.2

In [1]:
# Load WikiText-2 Dataset using Hugging Face datasets

import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from collections import Counter
import re

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load WikiText-2 dataset from Hugging Face
print("Loading WikiText-2 dataset from Hugging Face...")
dataset = load_dataset("wikitext", "wikitext-2-v1")

# Access the splits
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Show a sample
# Find a non-empty sample
print(f"\nLooking for non-empty samples...")
sample_text = None
for i, example in enumerate(train_dataset):
    if example['text'].strip():  # Find first non-empty line
        sample_text = example['text']
        print(f"Found non-empty sample at index {i}")
        break
print(f"First sample: {sample_text[:200]}")

# Basic tokenization function
def basic_tokenize(text):
    # Convert to lowercase and split on whitespace and punctuation
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

# Build vocabulary from training data
print("\nBuilding vocabulary...")
all_tokens = []
non_empty_lines = 0
for example in train_dataset:
    if example['text'].strip():  # Skip empty lines
        tokens = basic_tokenize(example['text'])
        all_tokens.extend(tokens)
        non_empty_lines += 1

# Create vocabulary
vocab_counter = Counter(all_tokens)
vocab_size = len(vocab_counter)
print(f"Non-empty lines: {non_empty_lines}")
print(f"Total tokens: {len(all_tokens)}")
print(f"Vocabulary size: {vocab_size}")

# Create word to index mapping
vocab = {'<unk>': 0, '<pad>': 1}
for word, count in vocab_counter.most_common():
    if word not in vocab:
        vocab[word] = len(vocab)

# Create index to word mapping
idx_to_word = {idx: word for word, idx in vocab.items()}

print(f"Final vocabulary size (with special tokens): {len(vocab)}")
print(f"Most common words: {list(vocab_counter.most_common(10))}")

# Function to convert text to indices
def text_to_indices(text, vocab):
    tokens = basic_tokenize(text)
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

# Convert a sample to see the tokenization
sample_indices = text_to_indices(sample_text, vocab)
print(f"\nSample tokenization:")
print(f"Original: {sample_text[:100]}")
print(f"Tokens: {basic_tokenize(sample_text)[:20]}")
print(f"Indices: {sample_indices[:20]}")

print("\nDataset loaded successfully!")

/home/archer/archer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Loading WikiText-2 dataset from Hugging Face...
Train dataset size: 36718
Validation dataset size: 3760
Test dataset size: 4358

Looking for non-empty samples...
Found non-empty sample at index 1
First sample:  = Valkyria Chronicles III = 


Building vocabulary...
Non-empty lines: 23767
Total tokens: 1750345
Vocabulary size: 28710
Final vocabulary size (with special tokens): 28712
Most common words: [('the', 130768), ('of', 57030), ('unk', 54625), ('and', 50735), ('in', 45015), ('to', 39521), ('a', 36523), ('was', 21008), ('on', 15140), ('as', 15058)]

Sample tokenization:
Original:  = Valkyria Chronicles III = 

Tokens: ['valkyria', 'chronicles', 'iii']
Indices: [3767, 3830, 860]

Dataset loaded successfully!


In [2]:
# Data Preprocess & Init DataLoader

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class WikiTextDataset(Dataset):
    def __init__(self, hf_dataset, vocab, sequence_length=50):
        """
        Custom Dataset for WikiText-2
        
        Args:
            hf_dataset: Hugging Face dataset split
            vocab: vocabulary dictionary
            sequence_length: length of input sequences
        """
        self.vocab = vocab
        self.sequence_length = sequence_length
        
        # Process all text and create sequences
        self.sequences = []
        self.targets = []
        
        print(f"Processing dataset with sequence length: {sequence_length}")
        
        # Concatenate all non-empty text and tokenize
        all_indices = []
        for example in hf_dataset:
            if example['text'].strip():  # Skip empty lines
                indices = text_to_indices(example['text'], vocab)
                all_indices.extend(indices)
        
        print(f"Total tokens in dataset: {len(all_indices)}")
        
        # Create sequences of specified length
        for i in range(len(all_indices) - sequence_length):
            # Input sequence
            seq = all_indices[i:i + sequence_length]
            # Target is the next token
            target = all_indices[i + sequence_length]
            
            self.sequences.append(seq)
            self.targets.append(target)
        
        print(f"Created {len(self.sequences)} sequences")
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return (
            torch.tensor(self.sequences[idx], dtype=torch.long),
            torch.tensor(self.targets[idx], dtype=torch.long)
        )

# Create datasets
sequence_length = 50
batch_size = 64

print("Creating training dataset...")
train_wiki_dataset = WikiTextDataset(train_dataset, vocab, sequence_length)

print("\nCreating validation dataset...")
val_wiki_dataset = WikiTextDataset(val_dataset, vocab, sequence_length)

print("\nCreating test dataset...")
test_wiki_dataset = WikiTextDataset(test_dataset, vocab, sequence_length)

# Create DataLoaders
train_loader = DataLoader(
    train_wiki_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0  # Set to 0 for compatibility
)

val_loader = DataLoader(
    val_wiki_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

test_loader = DataLoader(
    test_wiki_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

print(f"\nDataLoader Summary:")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")
print(f"Batch size: {batch_size}")
print(f"Sequence length: {sequence_length}")

# Test the DataLoader
print("\nTesting DataLoader...")
for batch_idx, (sequences, targets) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print(f"  Sequences shape: {sequences.shape}")
    print(f"  Targets shape: {targets.shape}")
    print(f"  Sample sequence: {sequences[0][:10].tolist()}")
    print(f"  Sample target: {targets[0].item()}")
    
    # Convert back to words for verification
    sample_words = [idx_to_word[idx.item()] for idx in sequences[0]]
    target_word = idx_to_word[targets[0].item()]
    print(f"  Sample words: {sample_words}")
    print(f"  Target word: {target_word}")
    break  # Only show first batch

print("\nDataLoader created successfully!")

Creating training dataset...
Processing dataset with sequence length: 50
Total tokens in dataset: 1750345
Created 1750295 sequences

Creating validation dataset...
Processing dataset with sequence length: 50
Total tokens in dataset: 183606
Created 183556 sequences

Creating test dataset...
Processing dataset with sequence length: 50
Total tokens in dataset: 206381
Created 206331 sequences

DataLoader Summary:
Training batches: 27349
Validation batches: 2869
Test batches: 3224
Batch size: 64
Sequence length: 50

Testing DataLoader...
Batch 1:
  Sequences shape: torch.Size([64, 50])
  Targets shape: torch.Size([64])
  Sample sequence: [2613, 2119, 14696, 5, 1506, 10808, 6, 2437, 2, 2078]
  Sample target: 14
  Sample words: ['communities', 'planning', 'commissions', 'and', 'elected', 'boards', 'in', '1982', 'the', 'survey', 'showed', 'an', 'inconclusive', 'preference', 'for', 'a', 'limited', 'access', 'highway', 'in', 'what', 'was', 'termed', 'a', 'gut', 'level', 'reaction', 'to', 'the', 

In [3]:
# GRU Language Model Implementation

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

class GRULanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=512, num_layers=2, dropout=0.2):
        """
        GRU-based Language Model for next word prediction
        
        Args:
            vocab_size: Size of vocabulary
            embedding_dim: Dimension of word embeddings
            hidden_dim: Hidden dimension of GRU
            num_layers: Number of GRU layers
            dropout: Dropout probability
        """
        super(GRULanguageModel, self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        
        # Word embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # GRU layer(s)
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )
        
        # Dropout layer
        self.dropout_layer = nn.Dropout(dropout)
        
        # Output projection layer
        self.output_projection = nn.Linear(hidden_dim, vocab_size)
        
        # Initialize weights
        self.init_weights()
    
    def init_weights(self):
        """Initialize model weights"""
        # Initialize embedding weights
        nn.init.uniform_(self.embedding.weight, -0.1, 0.1)
        
        # Initialize GRU weights
        for name, param in self.gru.named_parameters():
            if 'weight' in name:
                nn.init.orthogonal_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        # Initialize output layer
        nn.init.xavier_uniform_(self.output_projection.weight)
        nn.init.zeros_(self.output_projection.bias)
    
    def forward(self, input_ids, hidden=None):
        """
        Forward pass
        
        Args:
            input_ids: Input token indices [batch_size, seq_len]
            hidden: Hidden state from previous step
            
        Returns:
            logits: Output logits [batch_size, seq_len, vocab_size]
            hidden: Final hidden state
        """
        batch_size, seq_len = input_ids.size()
        
        # Word embeddings
        embeddings = self.embedding(input_ids)  # [batch_size, seq_len, embedding_dim]
        
        # GRU forward pass, 2 layers by default
        gru_output, hidden = self.gru(embeddings, hidden)  # [batch_size, seq_len, hidden_dim]
        
        # Apply dropout to GRU output
        gru_output = self.dropout_layer(gru_output)
        
        # Project to vocabulary size
        logits = self.output_projection(gru_output)  # [batch_size, seq_len, vocab_size]
        
        return logits, hidden
    
    def init_hidden(self, batch_size):
        """Initialize hidden state"""
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device)

# Model hyperparameters
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 256
num_layers = 2
dropout = 0.2
learning_rate = 0.0001

# Initialize model
model = GRULanguageModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout=dropout
)

# Move model to device
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=vocab['<pad>'])  # Ignore padding tokens
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Print model information
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nGRU Language Model Summary:")
print(f"Vocabulary size: {vocab_size:,}")
print(f"Embedding dimension: {embedding_dim}")
print(f"Hidden dimension: {hidden_dim}")
print(f"Number of layers: {num_layers}")
print(f"Dropout: {dropout}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model device: {next(model.parameters()).device}")

# Test the model with a sample batch
print(f"\nTesting model with sample batch...")
model.eval()
with torch.no_grad():
    # Get a sample batch
    sample_sequences, sample_targets = next(iter(train_loader))
    sample_sequences = sample_sequences.to(device)
    sample_targets = sample_targets.to(device)
    
    # Forward pass
    logits, hidden = model(sample_sequences)
    
    print(f"Input shape: {sample_sequences.shape}")
    print(f"Output logits shape: {logits.shape}")
    print(f"Hidden state shape: {hidden.shape}")
    print(f"Target shape: {sample_targets.shape}")
    
    # Calculate loss for the last token prediction
    last_token_logits = logits[:, -1, :]  # [batch_size, vocab_size]
    loss = criterion(last_token_logits, sample_targets)
    print(f"Sample loss: {loss.item():.4f}")
    
    # Get predictions
    predictions = torch.argmax(last_token_logits, dim=-1)
    accuracy = (predictions == sample_targets).float().mean()
    print(f"Sample accuracy: {accuracy.item():.4f}")

print(f"\nModel initialized successfully!")


GRU Language Model Summary:
Vocabulary size: 28,712
Embedding dimension: 128
Hidden dimension: 256
Number of layers: 2
Dropout: 0.2
Total parameters: 11,745,320
Trainable parameters: 11,745,320
Model device: cpu

Testing model with sample batch...
Input shape: torch.Size([64, 50])
Output logits shape: torch.Size([64, 50, 28712])
Hidden state shape: torch.Size([2, 64, 256])
Target shape: torch.Size([64])
Sample loss: 10.2650
Sample accuracy: 0.0000

Model initialized successfully!


In [4]:
# Train the GRU Language Model

import torch
import torch.nn as nn
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """Train the model for one epoch"""
    model.train()
    total_loss = 0
    total_samples = 0
    correct_predictions = 0
    
    # Progress bar
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1} Training')
    
    for batch_idx, (sequences, targets) in enumerate(pbar):
        # Move data to device
        sequences = sequences.to(device)
        targets = targets.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        logits, hidden = model(sequences)
        
        # Calculate loss (predict next token from last timestep)
        last_token_logits = logits[:, -1, :]  # [batch_size, vocab_size]
        loss = criterion(last_token_logits, targets)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Update weights
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        total_samples += targets.size(0)
        
        # Calculate accuracy
        predictions = torch.argmax(last_token_logits, dim=-1)
        correct_predictions += (predictions == targets).sum().item()
        
        # Update progress bar
        avg_loss = total_loss / (batch_idx + 1)
        accuracy = correct_predictions / total_samples
        pbar.set_postfix({
            'Loss': f'{avg_loss:.4f}',
            'Acc': f'{accuracy:.4f}'
        })
        
        # Print detailed stats every 1000 batches
        if (batch_idx + 1) % 1000 == 0:
            print(f'\nBatch {batch_idx+1}/{len(train_loader)}:')
            print(f'  Loss: {avg_loss:.4f}')
            print(f'  Accuracy: {accuracy:.4f}')
    
    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_samples
    
    return avg_loss, accuracy

def validate_epoch(model, val_loader, criterion, device, epoch):
    """Validate the model"""
    model.eval()
    total_loss = 0
    total_samples = 0
    correct_predictions = 0
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc=f'Epoch {epoch+1} Validation')
        
        for sequences, targets in pbar:
            # Move data to device
            sequences = sequences.to(device)
            targets = targets.to(device)
            
            # Forward pass
            logits, hidden = model(sequences)
            
            # Calculate loss
            last_token_logits = logits[:, -1, :]
            loss = criterion(last_token_logits, targets)
            
            # Statistics
            total_loss += loss.item()
            total_samples += targets.size(0)
            
            # Calculate accuracy
            predictions = torch.argmax(last_token_logits, dim=-1)
            correct_predictions += (predictions == targets).sum().item()
            
            # Update progress bar
            avg_loss = total_loss / len(val_loader) if len(val_loader) > 0 else 0
            accuracy = correct_predictions / total_samples
            pbar.set_postfix({
                'Val Loss': f'{avg_loss:.4f}',
                'Val Acc': f'{accuracy:.4f}'
            })
    
    avg_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_samples
    
    return avg_loss, accuracy

def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10):
    """Complete training loop"""
    
    # Training history
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    
    best_val_loss = float('inf')
    best_model_state = None
    
    print(f"Starting training for {num_epochs} epochs...")
    print(f"Training batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")
    print("-" * 60)
    
    start_time = time.time()
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        # Training
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
        
        # Validation
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device, epoch)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            print(f"✓ New best model saved! Val Loss: {val_loss:.4f}")
        
        # Record history
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        # Print epoch summary
        epoch_time = time.time() - epoch_start_time
        print(f"\nEpoch {epoch+1}/{num_epochs} Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        print(f"  Time: {epoch_time:.2f}s")
        print("-" * 60)
    
    total_time = time.time() - start_time
    print(f"\nTraining completed in {total_time:.2f}s")
    print(f"Best validation loss: {best_val_loss:.4f}")
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print("Loaded best model weights")
    
    return {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies,
        'best_val_loss': best_val_loss
    }

def plot_training_history(history):
    """Plot training curves"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss plot
    ax1.plot(history['train_losses'], label='Train Loss', color='blue')
    ax1.plot(history['val_losses'], label='Val Loss', color='red')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy plot
    ax2.plot(history['train_accuracies'], label='Train Accuracy', color='blue')
    ax2.plot(history['val_accuracies'], label='Val Accuracy', color='red')
    ax2.set_title('Training and Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Start training
print("🚀 Starting GRU Language Model Training...")
print(f"Model has {sum(p.numel() for p in model.parameters()):,} parameters")

# Training parameters
num_epochs = 10

# Train the model
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=num_epochs
)

# Plot results
plot_training_history(history)

# Save the trained model
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'history': history,
    'vocab': vocab,
    'model_config': {
        'vocab_size': vocab_size,
        'embedding_dim': embedding_dim,
        'hidden_dim': hidden_dim,
        'num_layers': num_layers,
        'dropout': dropout
    }
}, 'gru_language_model.pth')

print("\n✅ Training completed!")
print("📁 Model saved as 'gru_language_model.pth'")
print(f"🎯 Final validation accuracy: {history['val_accuracies'][-1]:.4f}")

🚀 Starting GRU Language Model Training...
Model has 11,745,320 parameters
Starting training for 10 epochs...
Training batches: 27349
Validation batches: 2869
------------------------------------------------------------


Epoch 1 Training:   1%|          | 253/27349 [03:24<6:04:33,  1.24it/s, Loss=8.4111, Acc=0.0715]


KeyboardInterrupt: 

In [ ]:
# Test the trained model

# Text Generation and Next Word Prediction

import torch
import torch.nn.functional as F

def predict_next_word(model, input_text, vocab, idx_to_word, device, temperature=1.0, top_k=None):
    """
    Predict the next word given an input text
    
    Args:
        model: Trained GRU model
        input_text: Input sentence as string
        vocab: Vocabulary dictionary
        idx_to_word: Index to word mapping
        device: Device to run inference on
        temperature: Sampling temperature (higher = more random)
        top_k: If specified, only sample from top k most likely tokens
    
    Returns:
        predicted_word: The predicted next word
        probabilities: Top 10 word probabilities
    """
    model.eval()
    
    with torch.no_grad():
        # Tokenize input text
        tokens = basic_tokenize(input_text)
        if len(tokens) == 0:
            return "<unk>", []
        
        # Convert to indices
        indices = [vocab.get(token, vocab['<unk>']) for token in tokens]
        
        # Pad or truncate to sequence length (50)
        sequence_length = 50
        if len(indices) < sequence_length:
            # Pad with <pad> tokens at the beginning
            indices = [vocab['<pad>']] * (sequence_length - len(indices)) + indices
        else:
            # Take the last 50 tokens
            indices = indices[-sequence_length:]
        
        # Convert to tensor
        input_tensor = torch.tensor([indices], dtype=torch.long).to(device)  # [1, seq_len]
        
        # Forward pass
        logits, hidden = model(input_tensor)
        
        # Get logits for the last token
        last_logits = logits[0, -1, :]  # [vocab_size]
        
        # Apply temperature
        if temperature != 1.0:
            last_logits = last_logits / temperature
        
        # Apply top-k filtering if specified
        if top_k is not None:
            top_k = min(top_k, last_logits.size(-1))
            top_k_logits, top_k_indices = torch.topk(last_logits, top_k)
            # Set all other logits to negative infinity
            filtered_logits = torch.full_like(last_logits, float('-inf'))
            filtered_logits[top_k_indices] = top_k_logits
            last_logits = filtered_logits
        
        # Convert to probabilities
        probabilities = F.softmax(last_logits, dim=-1)
        
        # Get top 10 predictions for display
        top_probs, top_indices = torch.topk(probabilities, 10)
        top_words = [(idx_to_word[idx.item()], prob.item()) for idx, prob in zip(top_indices, top_probs)]
        
        # Sample from the distribution
        predicted_idx = torch.multinomial(probabilities, 1).item()
        predicted_word = idx_to_word[predicted_idx]
        
        return predicted_word, top_words

def generate_text(model, seed_text, vocab, idx_to_word, device, num_words=10, temperature=1.0, top_k=50):
    """
    Generate multiple words continuing from seed text
    
    Args:
        model: Trained GRU model
        seed_text: Starting text
        vocab: Vocabulary dictionary
        idx_to_word: Index to word mapping
        device: Device to run inference on
        num_words: Number of words to generate
        temperature: Sampling temperature
        top_k: Top-k sampling parameter
    
    Returns:
        generated_text: Complete generated text
    """
    model.eval()
    
    current_text = seed_text
    generated_words = []
    
    print(f"Seed text: '{seed_text}'")
    print("Generating words:")
    
    for i in range(num_words):
        predicted_word, top_words = predict_next_word(
            model, current_text, vocab, idx_to_word, device, temperature, top_k
        )
        
        # Skip special tokens
        if predicted_word in ['<unk>', '<pad>']:
            predicted_word = top_words[1][0] if len(top_words) > 1 else "the"
        
        generated_words.append(predicted_word)
        current_text += " " + predicted_word
        
        print(f"  {i+1}. {predicted_word} (top candidates: {[w[0] for w in top_words[:3]]})")
    
    generated_text = seed_text + " " + " ".join(generated_words)
    return generated_text

def interactive_prediction(model, vocab, idx_to_word, device):
    """
    Interactive mode for testing the model
    """
    print("\n🤖 Interactive Text Prediction Mode")
    print("Enter a sentence and I'll predict the next word!")
    print("Type 'quit' to exit, 'generate' to generate multiple words")
    print("-" * 60)
    
    while True:
        user_input = input("\nEnter text: ").strip()
        
        if user_input.lower() == 'quit':
            print("Goodbye! 👋")
            break
        
        if user_input.lower() == 'generate':
            seed_text = input("Enter seed text for generation: ").strip()
            if seed_text:
                num_words = int(input("How many words to generate? (default 10): ") or "10")
                temperature = float(input("Temperature (0.1-2.0, default 1.0): ") or "1.0")
                
                print(f"\n🎯 Generating {num_words} words...")
                generated = generate_text(model, seed_text, vocab, idx_to_word, device, 
                                        num_words=num_words, temperature=temperature, top_k=50)
                print(f"\n📝 Complete text: '{generated}'")
            continue
        
        if user_input:
            print(f"\n🔍 Analyzing: '{user_input}'")
            
            # Predict next word
            predicted_word, top_words = predict_next_word(
                model, user_input, vocab, idx_to_word, device, temperature=1.0, top_k=50
            )
            
            print(f"🎯 Predicted next word: '{predicted_word}'")
            print(f"\n📊 Top 10 candidates:")
            for i, (word, prob) in enumerate(top_words, 1):
                print(f"  {i:2d}. {word:<15} ({prob:.4f})")
            
            # Show the complete sentence
            complete_sentence = user_input + " " + predicted_word
            print(f"\n📝 Complete sentence: '{complete_sentence}'")

# Test the model with some examples
print("🧪 Testing the trained GRU model...")

# Example sentences to test
test_sentences = [
    "The cat sat on the",
    "I love to eat",
    "The weather today is",
    "Machine learning is",
    "In the future we will",
    "The president of the United",
    "Scientists have discovered a new",
    "The book was very"
]

print("\n📋 Testing with example sentences:")
print("=" * 60)

for i, sentence in enumerate(test_sentences, 1):
    print(f"\n{i}. Input: '{sentence}'")
    
    predicted_word, top_words = predict_next_word(
        model, sentence, vocab, idx_to_word, device, temperature=0.8, top_k=50
    )
    
    print(f"   Predicted: '{predicted_word}'")
    print(f"   Top 3: {[w[0] for w in top_words[:3]]}")
    print(f"   Complete: '{sentence} {predicted_word}'")

# Start interactive mode
interactive_prediction(model, vocab, idx_to_word, device)

🧪 Testing the trained GRU model...

📋 Testing with example sentences:

1. Input: 'The cat sat on the'
   Predicted: 'unk'
   Top 3: ['first', 'unk', 'island']
   Complete: 'The cat sat on the unk'

2. Input: 'I love to eat'
   Predicted: 'the'
   Top 3: ['the', 'a', 'unk']
   Complete: 'I love to eat the'

3. Input: 'The weather today is'
   Predicted: 'generally'
   Top 3: ['a', 'the', 'also']
   Complete: 'The weather today is generally'

4. Input: 'Machine learning is'
   Predicted: 'most'
   Top 3: ['a', 'the', 'an']
   Complete: 'Machine learning is most'

5. Input: 'In the future we will'
   Predicted: 'have'
   Top 3: ['be', 'have', 'not']
   Complete: 'In the future we will have'

6. Input: 'The president of the United'
   Predicted: 'states'
   Top 3: ['states', 'kingdom', 'nations']
   Complete: 'The president of the United states'

7. Input: 'Scientists have discovered a new'
   Predicted: 'government'
   Top 3: ['year', 'version', 'role']
   Complete: 'Scientists have disco


Enter text:  Hello, world! I am Archer. What is your



🔍 Analyzing: 'Hello, world! I am Archer. What is your'
🎯 Predicted next word: 'way'

📊 Top 10 candidates:
   1. unk             (0.3377)
   2. first           (0.1444)
   3. own             (0.0813)
   4. love            (0.0381)
   5. most            (0.0293)
   6. best            (0.0262)
   7. name            (0.0258)
   8. use             (0.0219)
   9. last            (0.0216)
  10. great           (0.0179)

📝 Complete sentence: 'Hello, world! I am Archer. What is your way'



Enter text:  He does not want to drink this



🔍 Analyzing: 'He does not want to drink this'
🎯 Predicted next word: 'time'

📊 Top 10 candidates:
   1. unk             (0.1353)
   2. with            (0.0529)
   3. to              (0.0496)
   4. as              (0.0481)
   5. in              (0.0475)
   6. work            (0.0421)
   7. use             (0.0342)
   8. other           (0.0289)
   9. name            (0.0271)
  10. time            (0.0265)

📝 Complete sentence: 'He does not want to drink this time'


...


🔍 Analyzing: 'How are you, find and thank'
🎯 Predicted next word: 'when'

📊 Top 10 candidates:
   1. the             (0.1869)
   2. by              (0.1784)
   3. for             (0.0763)
   4. a               (0.0529)
   5. unk             (0.0462)
   6. and             (0.0384)
   7. in              (0.0366)
   8. as              (0.0358)
   9. to              (0.0271)
  10. that            (0.0182)

📝 Complete sentence: 'How are you, find and thank when'



Enter text:  How are you, fine and thank



🔍 Analyzing: 'How are you, fine and thank'
🎯 Predicted next word: 'richard'

📊 Top 10 candidates:
   1. by              (0.1482)
   2. the             (0.1446)
   3. unk             (0.1255)
   4. for             (0.0554)
   5. and             (0.0469)
   6. a               (0.0303)
   7. in              (0.0259)
   8. john            (0.0249)
   9. it              (0.0236)
  10. as              (0.0173)

📝 Complete sentence: 'How are you, fine and thank richard'



Enter text:  How are you, fine and thank you.



🔍 Analyzing: 'How are you, fine and thank you.'
🎯 Predicted next word: 'think'

📊 Top 10 candidates:
   1. have            (0.1389)
   2. are             (0.0899)
   3. and             (0.0662)
   4. unk             (0.0654)
   5. re              (0.0547)
   6. were            (0.0524)
   7. to              (0.0382)
   8. was             (0.0378)
   9. can             (0.0375)
  10. had             (0.0271)

📝 Complete sentence: 'How are you, fine and thank you. think'
